In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path
from torch.autograd import Variable


In [173]:
valid_error = {}
dataset =3
gid = 0
for fold_num in range(5):
    valid_error[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        valid_error[fold_num][appliance] = {}
        for lr in [0.001, 0.01, 0.1]:
            valid_error[fold_num][appliance][lr] = {}
            for iters in [200000]:
            
                directory = "../code/baseline/cnn-tree/{}/{}/{}/{}/0.0/".format(dataset, fold_num, lr, iters)
                filename = "test-error-[\'{}\'].npy".format(appliance)
                
                full_path = directory + filename
                my_file = Path(full_path)
                if not my_file.exists():
                    print("CUDA_VISIBLE_DEVICES={} python cnn-tree.py {} {} {} 0 {} {} &".format(gid, dataset, lr, iters, fold_num, appliance))
                    gid += 1
                    if gid == 4:
                        gid = 0
                else:
                    for it in range(1000, 20000+1, 1000):
                        valid_error[fold_num][appliance][lr][it] = np.load(full_path).item()[it][appliance]

In [189]:
best_param = {}
for fold_num in range(5):
    best_param[fold_num] = {}

for fold_num in range(5):
    error = 70
    for lr in [0.001, 0.01, 0.1]:
        for iters in range(1000, 20000+1, 1000):
            cur_error = 0
            for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
                cur_error += valid_error[fold_num][appliance][lr][iters]
            cur_error /= 5
            if fold_num == 3:
                
                if cur_error > error and cur_error < 99:
                    print(cur_error)
                    error = cur_error
                    best_param[fold_num]['lr'] = lr
                    best_param[fold_num]['iters'] = iters
            else:
                if cur_error > error and cur_error < 90:
                    print(cur_error)
                    error = cur_error
                    best_param[fold_num]['lr'] = lr
                    best_param[fold_num]['iters'] = iters

78.8538241115
82.6690777515
82.7557656392
83.4145010895
86.6381789034
88.8714361757
88.192769871
89.3891055712
98.8196484271
98.9605525049
89.4240824275
89.9697676735


In [190]:
best_param

{0: {'iters': 11000, 'lr': 0.01},
 1: {'iters': 1000, 'lr': 0.01},
 2: {'iters': 1000, 'lr': 0.01},
 3: {'iters': 7000, 'lr': 0.1},
 4: {'iters': 3000, 'lr': 0.01}}

In [191]:
best_error = {}
for fold_num in range(5):
    best_error[fold_num] = {}
    lr = best_param[fold_num]['lr']
    iters = best_param[fold_num]['iters']
    
    directory = "../code/baseline/cnn-tree/{}/{}/{}/200000/0.0/".format(dataset, fold_num, lr)
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        filename = "test-error-[\'{}\'].npy".format(appliance)
        full_path = directory + filename
        best_error[fold_num][appliance] = np.load(full_path).item()[iters][appliance]


In [192]:
def get_overall_error(best_error):
    error = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        error[appliance] = best_error[0][appliance]*14 + best_error[1][appliance]*14 + best_error[2][appliance]*14 + best_error[3][appliance]*13 + best_error[4][appliance]*13
        print(error[appliance])
        error[appliance] /= 68
    return error

In [193]:
pd.Series(get_overall_error(best_error))

22097.1268359
2193.99922947
4655.30054188
1066.93095191
584.366251013


dr         68.460302
dw         15.690161
fridge     32.264695
hvac      324.957748
mw          8.593621
dtype: float64

In [172]:
best_param

{0: {'iters': 11000, 'lr': 0.01},
 1: {'iters': 3000, 'lr': 0.001},
 2: {'iters': 2000, 'lr': 0.001},
 3: {'iters': 13000, 'lr': 0.01},
 4: {'iters': 6000, 'lr': 0.001}}

In [194]:
np.save("../code/baseline/result/cnn-individual-param-{}.npy".format(dataset), best_param)